In [ ]:
!JAVA_OPTS="$JAVA_OPTS -Xmx=8g -Xms=8"


from pyspark import SparkConf, SparkContext
sc.stop()
conf = (SparkConf()
 .set("spark.driver.memory", "8g")
 .set("spark.driver.maxResultSize", "8g")
 .set("spark.executor.memory", "10000mb")
 .set("spark.storage.memoryFraction", "0")
 .set("spark.executor.instances", "100"))
sc = SparkContext(conf=conf)

In [3]:
import json
from ds_utils.pipeline import client, mongo
import datetime
import numpy as np

In [4]:
def highlight_properties(x):
    if 'ai' in x and len(x['ai']) > 0:
        sport = x['ai'][0]['st']
        location = x['ai'][0]['sl']
        team = x['ai'][0]['t']
    else:
        sport = None
        location = None
        team = None
    highlight_sport = (x['_id'], sport, location, team)
    return highlight_sport

In [5]:
def get_sec(s):
    L = s.split(':')
    if len(L) == 1:
        return L[0]
    elif len(L) == 2:
        datee = datetime.datetime.strptime(s, "%M:%S")
        return datee.minute * 60 + datee.second
    elif len(L) == 3:
        datee = datetime.datetime.strptime(s, "%H:%M:%S")
        return datee.hour * 3600 + datee.minute * 60 + datee.second

In [6]:
def duration(x):
    s = 0
    if 'c' in x and len(x['c'])>0:
        for clip in x['c']:
            if type(clip['d']) == int:
                s = s + (int(clip['d'])/10000000)
            elif type(clip['d']) == unicode and len(clip['d']) == 8:
                s = s + get_sec(clip['d'])
            elif type(clip['d']) == unicode and len(clip['d']) == 16:
                s = s + get_sec(clip['d'][0:8])
            else:
                continue
    else:
        s = None
    return s

# Recommendations

In [39]:
recommendations = sc.textFile("s3://ds-fulla/mongo/20150507/monolith/highlights/userhighlights/")\
    .map(json.loads)\
    .filter(lambda x: x['p'] and 'hm' in x and 'xt' in x)\
    .filter(lambda x: 'ai' in x and len(x['ai']) > 0)

In [40]:
def format(x):
    loc = x[0][2]
    if ',' in loc:
        s = loc.split(', ')
        city = s[0]
        state = s[1]
    elif loc != '':
        city = loc
        state = ''
    else:
        city = ''
        state = ''
    d = {'id': x[0][0], 'sport' : x[0][1], 'city' : city, 'state' : state, 'team' : x[0][3], 'duration' : x[1]}
    return d

In [41]:
recommendation2 = recommendations.map(lambda x: (highlight_properties(x), duration(x)))\
    .map(format)
# ID, Sport, Location, Team, duration

In [43]:
recommendation2.count()

177935

In [44]:
PATH = os.path.join("s3://hudl-hadoop/ds-highlight-recommendations", 'temp_date', "intermediate_data/")
ALL_SPORTS_PATH = os.path.join(PATH, 'all_sports/')

In [45]:
# If failes, call:
# s3cmd rm -r s3://hudl-hadoop/ds-highlight-recommendations/
recommendation2 \
    .map(json.dumps) \
    .saveAsTextFile(ALL_SPORTS_PATH)

In [46]:
recommendation3 = sc.textFile(ALL_SPORTS_PATH).map(json.loads)

In [47]:
sport = {
    1:  'football',
    2:  'basketball',
    3:  'wrestling',
    4:  'volleyball',
    5:  'baseball',
    6:  'soccer',
    7:  'lacrosse',
    8:  'golf',
    9:  'gymnastics',
    10: 'softball',
    11: 'swimdive',
    12: 'trackfield',
    13: 'icehockey',
    14: 'fieldhockey',
    15: 'waterpolo',
    16: 'cheerspirit',
    17: 'dancedrill',
    18: 'cricket',
    19: 'crosscountry',
    20: 'performingarts',
    21: 'rugby',
    22: 'tennis',
    100: 'other'
}

In [50]:
for s in sport:
    recommendation4 = recommendation3.filter(lambda x: x['sport'] == s)
    
    SPORT_PATH = os.path.join(PATH, 'sports/', sport[s])
    
    recommendation4.map(json.dumps) \
        .saveAsTextFile(SPORT_PATH)

In [51]:
def lon_lat_range():
    min_lon = sqlCtx.sql("select min(double(longitude)) as min_lon from hudl_sql.schools") \
        .collect()[0] \
        .min_lon
    max_lon = sqlCtx.sql("select max(double(longitude)) as max_lon from hudl_sql.schools") \
        .collect()[0] \
        .max_lon
    min_lat = sqlCtx.sql("select min(double(latitude)) as min_lat from hudl_sql.schools") \
        .collect()[0] \
        .min_lat
    max_lat = sqlCtx.sql("select max(double(latitude)) as max_lat from hudl_sql.schools") \
        .collect()[0] \
        .max_lat
    return min_lon, max_lon, min_lat, max_lat

In [52]:
min_lon, max_lon, min_lat, max_lat = lon_lat_range()

location_query = """
select t.teamid,
    double(s.longitude) as longitude,
    double(s.latitude) as latitude
from hudl_sql.teams t
inner join schools s
on t.schoolid=s.schoolid
"""

location_vector_rdd = sqlCtx.sql(location_query) \
    .map(
        # (user_id, np.array([normalized_lon, normalized_lat]))
        lambda x: (
            x.teamid,
            np.array([
                (x.longitude - min_lon) / (max_lon - min_lon),
                (x.latitude - min_lat) / (max_lat - min_lat)
            ])
        )
    )

Py4JJavaError: An error occurred while calling o379.javaToPython.
: java.lang.OutOfMemoryError: PermGen space
	at java.lang.ClassLoader.defineClass1(Native Method)
	at java.lang.ClassLoader.defineClass(ClassLoader.java:800)
	at java.security.SecureClassLoader.defineClass(SecureClassLoader.java:142)
	at java.net.URLClassLoader.defineClass(URLClassLoader.java:449)
	at java.net.URLClassLoader.access$100(URLClassLoader.java:71)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:361)
	at java.net.URLClassLoader$1.run(URLClassLoader.java:355)
	at java.security.AccessController.doPrivileged(Native Method)
	at java.net.URLClassLoader.findClass(URLClassLoader.java:354)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:425)
	at sun.misc.Launcher$AppClassLoader.loadClass(Launcher.java:308)
	at java.lang.ClassLoader.loadClass(ClassLoader.java:358)
	at org.apache.spark.sql.execution.Aggregate.output(Aggregate.scala:61)
	at org.apache.spark.sql.execution.Aggregate$$anonfun$1$$anonfun$apply$1.a

In [ ]:
location_vector_rdd.first()

In [ ]:
def assign_group(x):
    if x[3] != "":
        x["group"] = x["state"]
    else:
        x["group"] = "OTHER"
    return x

In [ ]:
SPORT = 1;
recommendation5 = sc.textFile(os.path.join(PATH, 'sports/', sport[SPORT]))\
    .map(json.loads)\
    


In [ ]:
recommendation5.take(1)

In [ ]:
.map(assign_group).cache()